# Initialization

In [1]:
# Google Earth Engine packages
import ee
import geemap
import geemap.colormaps as cm

import xarray as xr
import numpy as np


# function to load nc file into GEE
def netcdf_to_ee(filename):
    ds = xr.open_dataset(filename)
    data = ds['z']

    lon_data = np.round(data['lon'], 3)
    lat_data = np.round(data['lat'], 3)

    dim_lon = np.unique(np.ediff1d(lon_data).round(3))
    dim_lat = np.unique(np.ediff1d(lat_data).round(3))

    if (len(dim_lon) != 1) or (len(dim_lat) != 1):
        print("The netCDF file is not a regular longitude/latitude grid")

    data_np = np.array(data)
    data_np = np.transpose(data_np)

    # Figure out if we need to roll the data or not
    # (see https://github.com/giswqs/geemap/issues/285#issuecomment-791385176)
    if np.max(lon_data) > 180:
        data_np = np.roll(data_np, 180, axis=0)
        west_lon = lon_data[0] - 180
    else:
        west_lon = lon_data[0]

    transform = [dim_lon[0], 0, float(west_lon) - dim_lon[0]/2, 0, dim_lat[0], float(lat_data[0]) - dim_lat[0]/2]

    image = geemap.numpy_to_ee(
        data_np, "EPSG:4326", transform=transform, band_names='z'
    )
    return image, data_np, transform

In [2]:
# initialize GEE at the beginning of session
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()         # authenticate when using GEE for the first time
    ee.Initialize()
    
Map = geemap.Map()

In [3]:
import configparser
import ast

# read local config.ini file
config = configparser.ConfigParser()
config.read('config.ini')

# get file config from config.ini
dir_input = config['FILE_SETTINGS']['DIR_INPUT']
dir_output = config['FILE_SETTINGS']['DIR_OUTPUT']
output_gpkg = dir_output + config['FILE_SETTINGS']['GPKG_NAME']
scenarios = config.getboolean('CONFIG', 'PROJECTIONS')

# Configure the downloaded date range

If you are only interested in modeling the past, set `PROJECTIONS=False` in the `config.ini` to download reanalysis data for your defined modeling period only. Otherwise, all available historic data (since 1979) is downloaded to provide the best possible basis for bias adjustment of the climate scenario data.

In [4]:
if scenarios == True:
    date_range = ['1979-01-01', '2022-01-01']
else:
    date_range = ast.literal_eval(config['CONFIG']['DATE_RANGE'])

***

# ERA5L Geopotential

### Load input data

Load ERA5L file for geopotential and add to Map

In [5]:
file = 'ERA5_land_Z_geopotential_HMA.nc'
filename = dir_input + file

In [6]:
image, data_np, transform = netcdf_to_ee(filename)

In [7]:
# add image as layer
vis_params =  {'min': int(data_np.min()), 'max': int(data_np.max()), 'palette': cm.palettes.terrain, 'opacity': 0.8}
Map.addLayer(image, vis_params, "ERA5L geopotential")

Load catchment and add to map

In [8]:
import geopandas as gpd

catchment_new = gpd.read_file(output_gpkg, layer='catchment_new')

catchment = geemap.geopandas_to_ee(catchment_new)
Map.addLayer(catchment, {'color': 'darkgrey'}, "Catchment")
Map.centerObject(catchment, zoom=9)

Show Map

In [9]:
Map

Map(center=[42.182797129005124, 78.18973871888807], controls=(WidgetControl(options=['position', 'transparent_…

### Calculate weighted average geopotential and convert to elevation

In [10]:
# execute reducer
dict = image.reduceRegion(ee.Reducer.mean(),
                          geometry=catchment,
                          crs='EPSG:4326',
                          crsTransform=transform)

# get mean value and print
mean_val = dict.getInfo()['z']
ele_dat = mean_val / 9.80665
print(f'geopotential mean: {mean_val:.2f}, elevation: {ele_dat:.2f}m.a.s.l.')

geopotential mean: 32711.74, elevation: 3335.67m.a.s.l.


***

### ERA5L Temperature and Precipitation Time Series

In [11]:
collection = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_RAW')\
    .select('temperature_2m','total_precipitation_sum')\
    .filterDate(date_range[0], date_range[1])

In [12]:
def setProperty(image):
    dict = image.reduceRegion(ee.Reducer.mean(), catchment)
    return image.set(dict)


withMean = collection.map(setProperty)

In [13]:
import pandas as pd
import datetime

df = pd.DataFrame()
df['ts'] = withMean.aggregate_array('system:time_start').getInfo()
df['temp'] = withMean.aggregate_array('temperature_2m').getInfo()
df['temp_c'] = df['temp'] - 273.15
df['prec'] = withMean.aggregate_array('total_precipitation_sum').getInfo()
df['prec'] = df['prec'] * 1000
df['dt'] = df['ts'].apply(lambda x: datetime.datetime.fromtimestamp(x / 1000))

In [14]:
df.to_csv(dir_output + 'ERA5L.csv',header=True,index=False)

In [15]:
display(df)

,ts,temp,temp_c,prec,dt
0,283996800000,257.392996,-15.757004,0.023400,1979-01-01 01:00:00
1,284083200000,256.435964,-16.714036,0.004129,1979-01-02 01:00:00
2,284169600000,257.867342,-15.282658,0.001601,1979-01-03 01:00:00
3,284256000000,258.322419,-14.827581,0.334886,1979-01-04 01:00:00
4,284342400000,258.056697,-15.093303,0.057215,1979-01-05 01:00:00
...,...,...,...,...,...
15701,1640563200000,258.974442,-14.175558,1.840898,2021-12-27 01:00:00
15702,1640649600000,257.128553,-16.021447,1.386033,2021-12-28 01:00:00
15703,1640736000000,255.763580,-17.386420,0.097033,2021-12-29 01:00:00
15704,1640822400000,257.308069,-15.841931,0.001715,2021-12-30 01:00:00


Append data reference elevation to settings.yml.

In [16]:
import yaml

def read_yaml(file_path):
    with open(file_path, 'r') as f:
        data = yaml.safe_load(f)
        return data
    
def write_yaml(data, file_path):
    with open(file_path, 'w') as f:
        yaml.safe_dump(data, f)

def update_yaml(file_path, new_items):
    data = read_yaml(file_path)
    data.update(new_items)
    write_yaml(data, file_path)

        
update_yaml(dir_output + 'settings.yml', {'ele_dat': float(ele_dat)})

# Ab hier wird es experimentell...

In [17]:
#collection = ee.ImageCollection('NASA/GDDP-CMIP6')\
#    .select('tas','pr')\
#    .filterDate('2020-01-01', '2020-12-31')\
#    .filter(ee.Filter.eq('model', 'ACCESS-CM2'))\
#    .filter(ee.Filter.eq('scenario', 'ssp245'))
    

In [18]:
#import pandas as pd
#import datetime

#withMean = collection.map(setProperty)

#df = pd.DataFrame()


In [19]:
#array = ee.Array([withMean.aggregate_array('tas'),withMean.aggregate_array('pr')])

In [20]:
#x = array.getInfo()

In [21]:
#df['ts'] = withMean.aggregate_array('system:time_start').getInfo()
#df['temp'] = withMean.aggregate_array('tas').getInfo()
#df['temp_corr_c'] = df['temp'] - 273.15
#df['prec'] = withMean.aggregate_array('pr').getInfo()
#df['dt'] = df['ts'].apply(lambda x: datetime.datetime.fromtimestamp(x / 1000))

In [22]:
#df.to_csv(dir_output + 'ACCESS-CM2_ssp245.csv',header=True,index=False)

In [23]:
#display(df)

In [24]:
#dataset = ee.ImageCollection('NASA/GDDP-CMIP6')\
#                .filter(ee.Filter.eq('model', 'ACCESS-CM2'))\
#                .filter(ee.Filter.date('2014-07-01', '2014-07-02'))
#minimumAirTemperature = dataset.select('tasmin')
#minimumAirTemperatureVis = {
#  'min': 240,
#  'max': 310,
#  'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
#}
#Map.setCenter(71, 52, 3)
#Map.addLayer(minimumAirTemperature, minimumAirTemperatureVis, 'Minimum Air Temperature (K)')

In [25]:
#Map

In [26]:
#dataset